In [ ]:
from IPython.display import Image
from IPython.core.display import HTML 
from os.path import basename, join, exists
import pickle,cv2, random, datetime, copy, collections, csv, sklearn, os, warnings
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import *
from tensorflow.keras.activations import sigmoid
from scipy.ndimage import zoom 
from sklearn import metrics
from tensorflow.keras import layers
from time import time
from numpy.random import seed
from skimage import exposure, transform
from skimage.transform import rotate
from scipy import ndimage, misc
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
tf.random.set_seed(42)
np.random.seed(42) 

In [ ]:
def conv2d_block(input_tensor, n_filters, kernel_size = 3, batchnorm = True):
    """Function to add 2 convolutional layers with the parameters passed to it"""
    #  kernel_regularizer='l1_l2'
    x1 = Conv2D(filters = n_filters, kernel_size = (3, 3),
                kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    x1 = BatchNormalization()(x1)
    x1 = tf.keras.layers.LeakyReLU()(x1)
    

    # second layer
    x2 = Conv2D(filters = n_filters, kernel_size = (3, 3),
                kernel_initializer = 'he_normal', padding = 'same')(x1)
    x2 = BatchNormalization()(x2)
    x2 = tf.keras.layers.LeakyReLU()(x2)
    
    x3 = SqueezeExcite(x2, n_filters//2)
    
    return x3

def SqueezeExcite(x, ratio=16):
    nb_chan = K.int_shape(x)[-1]

    y = layers.GlobalAveragePooling2D()(x)
    y = layers.Dense(nb_chan // ratio, activation=tf.keras.layers.LeakyReLU(alpha=0.3))(y)
    y = layers.Dense(nb_chan, activation='sigmoid')(y)

    y = layers.Multiply()([x, y])
    return y


def rrb_block2(input_tensor, n_filters, kernel_size = 3, batchnorm = True):
    """Function to add 2 convolutional layers with the parameters passed to it"""
    nb_chan = K.int_shape(input_tensor)[-1]
    x1 = Conv2D(filters = nb_chan, kernel_size=(3, 3), kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    x1 = BatchNormalization()(x1)
    x1 = tf.keras.layers.LeakyReLU()(x1)

    # second layer
    x2 = Conv2D(filters = nb_chan,kernel_size=(3, 3), kernel_initializer = 'he_normal', padding = 'same')(x1)
    x2 = BatchNormalization()(x2)
    x2 = tf.keras.layers.LeakyReLU()(x2)
    
    x3 = Conv2D(filters = nb_chan, kernel_size=(1, 1), kernel_initializer = 'he_normal', padding = 'same')(x2)
    x3 = BatchNormalization()(x3)
    x3 = tf.keras.layers.LeakyReLU()(x3)
    
    x4 = SqueezeExcite(x3, ratio=2)
    
    out1 = layers.Add()([x1, x2, x4])
    
    nb_chan = K.int_shape(x3)[-1]
    
    y = Conv2D(nb_chan, 1, activation = 'sigmoid', padding = 'same')(out1)
    
    y = layers.Multiply()([y, input_tensor])
        
    return y

def rrb_block1(input_tensor, n_filters, kernel_size = 3, batchnorm = True):
    """Function to add 2 convolutional layers with the parameters passed to it"""
    x1 = Conv2D(filters = n_filters, kernel_size=(3, 3), kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    x1 = BatchNormalization()(x1)
    x1 = tf.keras.layers.LeakyReLU()(x1)

    # second layer
    x2 = Conv2D(filters = n_filters,kernel_size=(3, 3), kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    x2 = BatchNormalization()(x2)
    x2 = tf.keras.layers.LeakyReLU()(x2)
    
    x3 = Conv2D(filters = n_filters, kernel_size=(1, 1), kernel_initializer = 'he_normal', padding = 'same')(input_tensor)

    out = layers.Add()([x1, x2, x3])
    
    return out

In [ ]:
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, MaxPooling2D, Concatenate

def ASPP_block(inputs):
    """Implementation of Atrous Spatial Pyramid Pooling block"""
    
    # Define the atrous convolution rates
    rates = [3, 6, 9]
    
    # Apply 1x1 convolution
    conv_1x1 = Conv2D(filters=64, kernel_size=1, activation='relu')(inputs)
    
    # Apply 3x3 atrous convolutions with different rates
    conv_3x3_rate_1 = Conv2D(filters=64, kernel_size=3, dilation_rate=rates[0], padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.3))(inputs)
    conv_3x3_rate_2 = Conv2D(filters=64, kernel_size=3, dilation_rate=rates[1], padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.3))(inputs)
    conv_3x3_rate_3 = Conv2D(filters=64, kernel_size=3, dilation_rate=rates[2], padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.3))(inputs)
    
    # Concatenate the output of all convolutions
    concat = Concatenate()([conv_1x1, conv_3x3_rate_1, conv_3x3_rate_2, conv_3x3_rate_3])
    
    # Apply 1x1 convolution
    aspp_out = Conv2D(filters=64, kernel_size=1, padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.3))(concat)
    
    return aspp_out


def res_block(x, n_filters, count):
    if count == 4:
        
        x1 = Conv2D(filters = n_filters, kernel_size = (3, 3),kernel_initializer = 'he_normal', padding = 'same')(x)
        x1 = BatchNormalization()(x1)
        x1 = tf.keras.layers.LeakyReLU()(x1)
        x2 = Conv2D(filters = n_filters, kernel_size = (1, 1),kernel_initializer = 'he_normal', padding = 'same')(x)
        x2 = BatchNormalization()(x2)
        x2 = tf.keras.layers.LeakyReLU()(x2)
        x3 = Add()([x1, x2])
        
        x4 = Conv2D(filters = n_filters, kernel_size = (3, 3),kernel_initializer = 'he_normal', padding = 'same')(x3)
        x4 = BatchNormalization()(x4)
        x4 = tf.keras.layers.LeakyReLU()(x4)
        x5 = Conv2D(filters = n_filters, kernel_size = (1, 1),kernel_initializer = 'he_normal', padding = 'same')(x3)
        x5 = BatchNormalization()(x5)
        x5 = tf.keras.layers.LeakyReLU()(x5)
        x6 = Add()([x4, x5])
        
        x7 = Conv2D(filters = n_filters, kernel_size = (3, 3),kernel_initializer = 'he_normal', padding = 'same')(x6)
        x7 = BatchNormalization()(x7)
        x7 = tf.keras.layers.LeakyReLU()(x7)
        x8 = Conv2D(filters = n_filters, kernel_size = (1, 1),kernel_initializer = 'he_normal', padding = 'same')(x6)
        x8 = BatchNormalization()(x8)
        x8 =tf.keras.layers.LeakyReLU()(x8)
        x9 = Add()([x7, x8])
        
        x10 = Conv2D(filters = n_filters, kernel_size = (3, 3),kernel_initializer = 'he_normal', padding = 'same')(x9)
        x10 = BatchNormalization()(x10)
        x10 = tf.keras.layers.LeakyReLU()(x10)
        x11 = Conv2D(filters = n_filters, kernel_size = (1, 1),kernel_initializer = 'he_normal', padding = 'same')(x9)
        x11 = BatchNormalization()(x11)
        x11 = tf.keras.layers.LeakyReLU()(x11)
        x12 = Add()([x10, x11])
        
        x13 = SqueezeExcite(x12, n_filters//4)
        return x13
    
    elif count == 3:
        
        x1 = Conv2D(filters = n_filters, kernel_size = (3, 3),kernel_initializer = 'he_normal', padding = 'same')(x)
        x1 = BatchNormalization()(x1)
        x1 = tf.keras.layers.LeakyReLU()(x1)
        x2 = Conv2D(filters = n_filters, kernel_size = (1, 1),kernel_initializer = 'he_normal', padding = 'same')(x)
        x2 = BatchNormalization()(x2)
        x2 = tf.keras.layers.LeakyReLU()(x2)
        x3 = Add()([x1, x2])
        
        x4 = Conv2D(filters = n_filters, kernel_size = (3, 3),kernel_initializer = 'he_normal', padding = 'same')(x3)
        x4 = BatchNormalization()(x4)
        x4 = tf.keras.layers.LeakyReLU()(x4)
        x5 = Conv2D(filters = n_filters, kernel_size = (1, 1),kernel_initializer = 'he_normal', padding = 'same')(x3)
        x5 = BatchNormalization()(x5)
        x5 = tf.keras.layers.LeakyReLU()(x5)
        x6 = Add()([x4, x5])
        
        x7 = Conv2D(filters = n_filters, kernel_size = (3, 3),kernel_initializer = 'he_normal', padding = 'same')(x6)
        x7 = BatchNormalization()(x7)
        x7 = tf.keras.layers.LeakyReLU()(x7)
        x8 = Conv2D(filters = n_filters, kernel_size = (1, 1),kernel_initializer = 'he_normal', padding = 'same')(x6)
        x8 = BatchNormalization()(x8)
        x8 = tf.keras.layers.LeakyReLU()(x8)
        x9 = Add()([x7, x8])
        
        x10 = SqueezeExcite(x9, n_filters//4)
        
        return x10
    
    elif count == 2:
        
        x1 = Conv2D(filters = n_filters, kernel_size = (3, 3),kernel_initializer = 'he_normal', padding = 'same')(x)
        x1 = BatchNormalization()(x1)
        x1 = tf.keras.layers.LeakyReLU()(x1)
        x2 = Conv2D(filters = n_filters, kernel_size = (1, 1),kernel_initializer = 'he_normal', padding = 'same')(x)
        x2 = BatchNormalization()(x2)
        x2 = tf.keras.layers.LeakyReLU()(x2)
        x3 = Add()([x1, x2])
        
        x4 = Conv2D(filters = n_filters, kernel_size = (3, 3),kernel_initializer = 'he_normal', padding = 'same')(x3)
        x4 = BatchNormalization()(x4)
        x4 = tf.keras.layers.LeakyReLU()(x4)
        x5 = Conv2D(filters = n_filters, kernel_size = (1, 1),kernel_initializer = 'he_normal', padding = 'same')(x3)
        x5 = BatchNormalization()(x5)
        x5 = tf.keras.layers.LeakyReLU()(x5)
        x6 = Add()([x4, x5])
        
        x7 = SqueezeExcite(x6, n_filters//4)
        
        return x7
    elif count == 1:
        
        x1 = Conv2D(filters = n_filters, kernel_size = (3, 3),kernel_initializer = 'he_normal', padding = 'same')(x)
        x1 = BatchNormalization()(x1)
        x1 = tf.keras.layers.LeakyReLU()(x1)
        x2 = Conv2D(filters = n_filters, kernel_size = (1, 1),kernel_initializer = 'he_normal', padding = 'same')(x)
        x2 = BatchNormalization()(x2)
        x2 = tf.keras.layers.LeakyReLU()(x2)
        x3 = Add()([x1, x2])
        
        x4 = SqueezeExcite(x3, n_filters//4)
        
        return x4

In [ ]:
from tensorflow.keras.layers import ZeroPadding2D,BatchNormalization,MaxPooling2D, UpSampling2D,Dropout,multiply, Concatenate, Conv2D, Add, Activation

def UNet_Seg(input_img, n_filters = 32, dropout = 0.2, batchnorm = True):
# set image specifics
    k = 3 # kernel size
    img_ch = 3 # image channels
    out_ch = 1 # output channel

    c1 = conv2d_block(input_img, n_filters * 1)
    p1 = layers.MaxPooling2D((2, 2))(c1)
    p1 = layers.Dropout(dropout)(p1)

    c2 = conv2d_block(p1, n_filters * 2)
    p2 = layers.MaxPooling2D((2, 2))(c2)
    p2 = layers.Dropout(dropout)(p2)
    
    c11 = layers.MaxPooling2D((4, 4))(c1)
    c3 = conv2d_block(p2, n_filters * 4)
    c31 = layers.Concatenate()([c3, res_block(c11, n_filters * 4, 4)])
    c32 = rrb_block2(c31, n_filters * 4, kernel_size=3, batchnorm=batchnorm)
    p3 = layers.MaxPooling2D((2, 2))(c32)
    p3 = layers.Dropout(dropout)(p3)
    
    
    c22 = layers.MaxPooling2D((4, 4))(c2)    
    c4 = conv2d_block(p3, n_filters * 6)
    c41 = layers.Concatenate()([c4, res_block(c22, n_filters * 6, 3)])
    c42 = rrb_block2(c41, n_filters * 4, kernel_size=3, batchnorm=batchnorm)
    p4 = layers.MaxPooling2D((2, 2))(c42)
    p4 = layers.Dropout(dropout)(p4)


    # D1
    c51 = ASPP_block(p4)
    c52 = Dropout(dropout)(c51)


    # Decoder path
    u6 = layers.UpSampling2D(size=(2, 2))(c52)
    u62 = layers.Concatenate()([u6, res_block(c4, n_filters * 6, 1)])
    u63 = conv2d_block(u62, n_filters * 6)
    u64 = rrb_block2(u63, n_filters * 6, kernel_size=3, batchnorm=batchnorm)
    d65 = Dropout(dropout)(u64)


    
    u7 = layers.UpSampling2D(size=(2, 2))(d65)
    u72 = layers.Concatenate()([u7, res_block(c3,n_filters * 4, 2)])
    u73 = conv2d_block(u72, n_filters * 4)
    u74 = rrb_block2(u73, n_filters * 4, kernel_size=3, batchnorm=batchnorm)
    d75 = Dropout(dropout)(u74)

    
    ### Decoder Full_Scale Up_Conv ####
    f1 = layers.UpSampling2D(size=(4, 4))(d65)
    ####################################
    
    u8 = layers.UpSampling2D(size=(2, 2))(d75)
    u82 = layers.Concatenate()([u8, f1, res_block(c2, n_filters * 2, 3)])
    u83 = conv2d_block(u82, n_filters * 2)
    u84 = rrb_block2(u83, n_filters * 2, kernel_size=3, batchnorm=batchnorm)
    d85 = Dropout(dropout)(u84)

    
    ### Decoder Full_Scale Up_Conv ####
    f2 = layers.UpSampling2D(size=(4, 4))(d75)
    #####################################
    
    
    u9 = layers.UpSampling2D(size=(2, 2))(d85)
    u92 = layers.Concatenate()([u9, f2, res_block(c1, n_filters * 1, 4)])
    u93 = conv2d_block(u92, n_filters * 1)
    u94 = rrb_block2(u93, n_filters * 1, kernel_size=3, batchnorm=batchnorm)
    d95 = Dropout(dropout)(u94)

   

    output1 = Conv2D(out_ch, (1, 1), padding='same', activation='sigmoid', name="out1")(d95)


    model = tf.keras.Model(inputs=input_img, outputs=output1)
    
    return model

In [ ]:
from tensorflow.keras.applications import MobileNetV2, VGG19, VGG16
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
  

def build_imagenet_unet(input_shape, n_filters=32, dropout=0.5, batchnorm=True):
    
    inputs = tf.keras.Input(shape=input_shape, name="input_image")
    
    encoder =  VGG19(input_tensor=inputs, weights="imagenet", include_top=False)
#     encoder.summary()    
    
    
    c50 = encoder.get_layer("block5_conv4").output
    c51 = ASPP_block(c50)
    c52 = Dropout(dropout)(c51)



    # Decoder path
    u6 = layers.UpSampling2D(size=(2, 2))(c52)
    u62 = layers.Concatenate()([u6, res_block(encoder.get_layer("block4_conv4").output, n_filters * 6, 1)])
    u63 = conv2d_block(u62, n_filters * 6)
    u64 = rrb_block2(u63, n_filters * 6, kernel_size=3, batchnorm=batchnorm)
    d65 = Dropout(dropout)(u64)
    
    
    u7 = layers.UpSampling2D(size=(2, 2))(d65)
    u72 = layers.Concatenate()([u7, res_block(encoder.get_layer("block3_conv4").output,n_filters * 4, 2)])
    u73 = conv2d_block(u72, n_filters * 4)
    u74 = rrb_block2(u73, n_filters * 4, kernel_size=3, batchnorm=batchnorm)
    d75 = Dropout(dropout)(u74)

    
    
    ### Decoder Full_Scale Up_Conv ####
    f1 = layers.UpSampling2D(size=(4, 4))(d65)
    ####################################
    
    
    
    u8 = layers.UpSampling2D(size=(2, 2))(d75)
    u82 = layers.Concatenate()([u8, f1, res_block(encoder.get_layer("block2_conv2").output, n_filters * 2, 3)])
    u83 = conv2d_block(u82, n_filters * 2)
    u84 = rrb_block2(u83, n_filters * 2, kernel_size=3, batchnorm=batchnorm)
    d85 = Dropout(dropout)(u84)


    
    ### Decoder Full_Scale Up_Conv ####
    f2 = layers.UpSampling2D(size=(4, 4))(d75)
    #####################################
    u9 = layers.UpSampling2D(size=(2, 2))(d85)
    u92 = layers.Concatenate()([u9, f2, res_block(encoder.get_layer("block1_conv2").output, n_filters * 1, 4)])
    u93 = conv2d_block(u92, n_filters * 1)
    u94 = rrb_block2(u93, n_filters * 1, kernel_size=3, batchnorm=batchnorm)
    d95 = Dropout(dropout)(u94)
    

    outputs = Conv2D(1, (1, 1), padding='same', activation='sigmoid', name="out1")(d95)
    
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    return model


In [ ]:
input_shape = (512, 512, 3)
model = build_imagenet_unet(input_shape, n_filters=16, dropout=0.2, batchnorm=True)

model.compile(optimizer=tf.keras.optimizers.Adam(3e-4),
              loss= weighted_bce_dice_loss,
              metrics=[dice_acc,jacard_acc,"accuracy"])

In [ ]:
def dice_acc(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
 
# jacard accuracy
def jacard_acc(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0)

def dice_loss(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = y_true_f * y_pred_f
    score = (2. * K.sum(intersection) + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return 1. - score


def tversky(y_true, y_pred, smooth=1, alpha=0.7):
    y_true_f = K.cast(y_true, 'float32')
    y_pred_f = K.cast(y_pred, 'float32')
    y_true_pos = K.flatten(y_true_f)
    y_pred_pos = K.flatten(y_pred_f)
    true_pos = K.sum(y_true_pos * y_pred_pos)
    false_neg = K.sum(y_true_pos * (1 - y_pred_pos))
    false_pos = K.sum((1 - y_true_pos) * y_pred_pos)
    return (true_pos + smooth) / (true_pos + alpha * false_neg + (1 - alpha) * false_pos + smooth)


def tversky_loss(y_true, y_pred):
    return 1 - tversky(y_true, y_pred)


def focal_tversky_loss(y_true, y_pred, gamma=0.75):
    tv = tversky(y_true, y_pred)
    return K.pow((1 - tv), gamma)


def weighted_bce_loss(y_true, y_pred, weight):
    # avoiding overflow
    epsilon = 1e-7
    y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
    logit_y_pred = K.log(y_pred / (1. - y_pred))

    # https://www.tensorflow.org/api_docs/python/tf/nn/weighted_cross_entropy_with_logits
    loss = (1. - y_true) * logit_y_pred + (1. + (weight - 1.) * y_true) * \
           (K.log(1. + K.exp(-K.abs(logit_y_pred))) + K.maximum(-logit_y_pred, 0.))
    return K.sum(loss) / K.sum(weight)


def weighted_dice_loss(y_true, y_pred, weight):
    smooth = 1.
    w, m1, m2 = weight * weight, y_true, y_pred
    intersection = (m1 * m2)
    score = (2. * K.sum(w * intersection) + smooth) / (K.sum(w * m1) + K.sum(w * m2) + smooth)
    loss = 1. - K.sum(score)
    return loss


def weighted_bce_dice_loss(y_true, y_pred):
    y_true = K.cast(y_true, 'float32')
    y_pred = K.cast(y_pred, 'float32')
    # if we want to get same size of output, kernel size must be odd number
    averaged_mask = K.pool2d(
        y_true, pool_size=(1, 1), strides=(1, 1), padding='same', pool_mode='avg')
    border = K.cast(K.greater(averaged_mask, 0.15), 'float32') * K.cast(K.less(averaged_mask, 0.85), 'float32')
    weight = K.ones_like(averaged_mask)
    w0 = K.sum(weight)
    weight += border * 2
    w1 = K.sum(weight)
    weight *= (w0 / w1)
#     loss = weighted_bce_loss(y_true, y_pred, weight) + focal_tversky_loss(y_true, y_pred) + weighted_dice_loss(y_true, y_pred, weight)
    loss = weighted_bce_loss(y_true, y_pred, weight) + dice_loss(y_true, y_pred)
    return loss

def loss(y_true, y_pred):
    def dice_loss(y_true, y_pred):
        numerator = 2 * tf.reduce_sum(y_true * y_pred, axis=(1,2,3))
        denominator = tf.reduce_sum(y_true + y_pred, axis=(1,2,3))

        return tf.reshape(1 - numerator / denominator, (-1, 1, 1))

    return binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)

In [ ]:
def calc_DSC(truth, pred, classes=2):
    dice_scores = []
    # Iterate over each class
    for i in range(classes):
        try:
            gt = np.equal(truth, i)
            pd = np.equal(pred, i)
            # Calculate Dice
            dice = 2*np.logical_and(pd, gt).sum() / (pd.sum() + gt.sum())
            dice_scores.append(dice)
        except ZeroDivisionError:
            dice_scores.append(0.0)
    # Return computed Dice Similarity Coefficients
    return dice_scores[1]

def calc_IoU(truth, pred, classes):
    iou_scores = []
    # Iterate over each class
    for i in range(classes):
        try:
            gt = np.equal(truth, i)
            pd = np.equal(pred, i)
            # Calculate iou
            iou = np.logical_and(pd, gt).sum() / (pd.sum() + gt.sum() - np.logical_and(pd, gt).sum())
            iou_scores.append(iou)
        except ZeroDivisionError:
            iou_scores.append(0.0)
    # Return computed IoU
    return iou_scores[1]


def calc_Sensitivity(truth, pred, classes):
    sens_scores = []
    # Iterate over each class
    for i in range(classes):
        try:
            gt = np.equal(truth, i)
            pd = np.equal(pred, i)
            # Calculate sensitivity
            sens = np.logical_and(pd, gt).sum() / gt.sum()
            sens_scores.append(sens)
        except ZeroDivisionError:
            sens_scores.append(0.0)
    # Return computed sensitivity scores
    return sens_scores[1]

def calc_Specificity(truth, pred, classes):
    spec_scores = []
    # Iterate over each class
    for i in range(classes):
        try:
            not_gt = np.logical_not(np.equal(truth, i))
            not_pd = np.logical_not(np.equal(pred, i))
            # Calculate specificity
            spec = np.logical_and(not_pd, not_gt).sum() / (not_gt).sum()
            spec_scores.append(spec)
        except ZeroDivisionError:
            spec_scores.append(0.0)
    # Return computed specificity scores
    return spec_scores[1]

def calc_Accuracy(truth, pred, classes):
    acc_scores = []
    # Iterate over each class
    for i in range(classes):
        try:
            gt = np.equal(truth, i)
            pd = np.equal(pred, i)
            not_gt = np.logical_not(np.equal(truth, i))
            not_pd = np.logical_not(np.equal(pred, i))
            # Calculate accuracy
            acc = (np.logical_and(pd, gt).sum() + \
                   np.logical_and(not_pd, not_gt).sum()) /  gt.size
            acc_scores.append(acc)
        except ZeroDivisionError:
            acc_scores.append(0.0)
    # Return computed accuracy scores
    return acc_scores[1]

def calc_Precision(truth, pred, classes):
    prec_scores = []
    # Iterate over each class
    for i in range(classes):
        try:
            gt = np.equal(truth, i)
            pd = np.equal(pred, i)
            # Calculate precision
            prec = np.logical_and(pd, gt).sum() / pd.sum()
            prec_scores.append(prec)
        except ZeroDivisionError:
            prec_scores.append(0.0)
    # Return computed precision scores
    return prec_scores[1]

In [ ]:
OUT_DIR = "./preprocessed_np"

x_train = np.load(os.path.join(OUT_DIR, "x_train.npy")) 
y_train = np.load(os.path.join(OUT_DIR, "y_train.npy"))   
x_test  = np.load(os.path.join(OUT_DIR, "x_test.npy"))
y_test  = np.load(os.path.join(OUT_DIR, "y_test.npy"))


In [ ]:

earlystop = EarlyStopping(monitor="val_dice_acc", patience=20, verbose=1, mode="max", restore_best_weights=True)
ReduceLROnPlat = ReduceLROnPlateau(
    monitor="val_dice_acc", factor=0.1, patience=20, cooldown=3, min_lr=0, verbose=1, min_delta=0.0001
)

callbacks = [earlystop, ReduceLROnPlat, ClearMemory(), model_checkpoint]


start = time()

history = model.fit(
    x_train, y_train,
    epochs=200,
    batch_size=global_batch_size,
    verbose=1,
    shuffle=True,
    callbacks=callbacks,
    validation_data=(x_test, y_test)
)

end = time()
print(f"\nModel took {end - start:.2f} seconds to train")
print(f"Model took {(end - start)/60:.2f} minutes to train\n")

results = model.predict(x_test, verbose=1)

In [ ]:
pred_m = copy.deepcopy(results)
act = copy.deepcopy(y_test)

t = 0.5

pred_m[pred_m >= t] = 1
pred_m[pred_m < t] = 0

act[act >= t] = 1
act[act < t] = 0

for x in range(pred_m.shape[0]):
    tc = pred_m[x].flatten()
    tc = collections.Counter(tc)
    tv = tc[1]
    if tv < 10000:
        pred_m[x][pred_m[x] == 1] = 0

dice_v = round(calc_DSC(act, pred_m, 2),4)
iou_v = round(calc_IoU(act, pred_m, 2),4)
sens_v = round(calc_Sensitivity(act, pred_m, 2),4)
spec_v = round(calc_Specificity(act, pred_m, 2),4)
prec_v = round(calc_Precision(act, pred_m, 2),4)
auc_v = round(metrics.roc_auc_score(act.ravel(), pred_m.ravel()),4)
acc_v = round(calc_Accuracy(act, pred_m, 2),4)

print("Accuracy = ", acc_v)
print("Precision = ", prec_v)
print("F1/Dice = ", dice_v)
print("IoU = ", iou_v)
print("Sensitivity = ", sens_v)
print("Specificity = ", spec_v)